This Jupyter Notebook is part of the course [Python for Industry 4.0](https://www.udemy.com/course/python-for-industry-40/?referralCode=D7925A2D76BA4C94CA4E) from [Industry 4.0 Academy](https://www.i40a.com).

Latos© copyright 2022. All Rights Reserved.

# Project 1 - OEE Calculation

## OEE (Wikipedia)
Overall Equipment Effectiveness (OEE) is a measure of how well a manufacturing operation is utilized (facilities, time and material) compared to its full potential, during the periods when it is scheduled to run. It identifies the percentage of manufacturing time that is truly productive. An OEE of 100% means that only good parts are produced (100% quality), at the maximum speed (100% performance), and without interruption (100% availability).


### Calculations for OEE
The OEE of a manufacturing unit are calculated as the product of three separate components:

* **Availability**: percentage of scheduled time that the operation is available to operate. Often referred to as Uptime.

* **Quality**: Good Units produced as a percentage of the Total Units Started.

* **Performance**: speed at which the Work Center runs as a percentage of its designed speed.


To learn more about calculating OEE, see the article on [Wikipedia](https://en.wikipedia.org/wiki/Overall_equipment_effectiveness).

## Aim of the project
Build a Python function to calculate the OEE based on a given dataset.


## Dataset
Complete information about the dataset is available on the [dataset repository](https://github.com/i40a/datasets/blob/main/oee_simulation/info.md).



## Load the dataset

Load the csv file on the path below

In [ ]:
import pandas as pd
path = 'https://media.githubusercontent.com/media/i40a/datasets/main/oee_simulation/original/oee.csv'

In [ ]:
# load csv file
df = pd.read_csv(path, index_col='Timestamp', parse_dates=True)
df.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:00:00.000000,state,idle,line_1,0.892188
2022-07-01 00:00:00.000000,bad_count,1,line_2,6.932149
2022-07-01 00:00:00.000000,good_count,21,line_2,6.932149
2022-07-01 00:00:00.000000,state,normal,line_2,6.932149
2022-07-01 00:00:53.531259,state,idle,line_1,1.409600


## Preprocessing

In order to save the entire dataset in a single CSV file, three different measurements are concateneted in a single table. 


To better analyse and work with the dataset, we are going to start the project by spliting the dataset into two DataFrames:

* df_state - DataFrame with the state of the plant. The type of the measurement is string. Tip: 'Measurement' == 'state'

* df_count - DataFrame with the counter of good and bad units. The type of the measurement is integer. Tip: 'Measurement'!= 'state'



In [ ]:
# create the df_state DataFrame from df
df_state = df[df['Measurement']=='state']
df_state.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:00:00.000000,state,idle,line_1,0.892188
2022-07-01 00:00:00.000000,state,normal,line_2,6.932149
2022-07-01 00:00:53.531259,state,idle,line_1,1.409600
2022-07-01 00:02:18.107264,state,idle,line_1,1.324097
2022-07-01 00:03:37.553060,state,normal,line_1,12.179597


In [ ]:
# create the df_count DataFrame from df and change type of the Value column to integer
df_count = df[df['Measurement']!='state']
df_count['Value'] = df_count['Value'].astype(int)
df_count.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:00:00.000000,bad_count,1,line_2,6.932149
2022-07-01 00:00:00.000000,good_count,21,line_2,6.932149
2022-07-01 00:03:37.553060,good_count,37,line_1,12.179597
2022-07-01 00:03:37.553060,bad_count,1,line_1,12.179597
2022-07-01 00:08:38.227734,bad_count,2,line_2,14.268206


## Explore the dataset
First, understand the data and variables from the [dataset repository](https://github.com/i40a/datasets/blob/main/oee_simulation/info.md).

Then, use the numerical and visualization libraries to get insights from the data.

### Possible operations
* Number and name of the production line
* Plot the states over time
* Plot the production of good and bad units over time

In [ ]:
# check production lines
df_state['Line'].value_counts()

line_1    7049
line_2    5675
Name: Line, dtype: int64

In [ ]:
# check production lines
df_state['Value'].value_counts()

normal          6314
idle            6184
planned_stop     204
fault             22
Name: Value, dtype: int64

In [ ]:
df_count['Measurement'].value_counts()

bad_count     6314
good_count    6314
Name: Measurement, dtype: int64

In [ ]:
# plot state over time
import plotly.express as px
fig = px.line(df_state, x=df_state.index, y='Value', color='Line', line_shape='hv')
fig.show()

In [ ]:
# plot counters over time
line = 'line_1'

df_count_line = df_count[df_count['Line']==line]

fig = px.line(df_count_line, x=df_count_line.index, y='Value', color='Measurement')
fig.show()

## OEE Calculation

### Availability

run_time = time normal – The machine's total time running without issue.

down_time = time idle + time fault + time planned_stop – The machine's total downtime, calculated as the sum of the machine states other than run_time.

availability = run_time / (run_time + down_time) – The machine's uptime or percentage of scheduled time that the machine is available to operate over the specified time interval.


Aim: build a function that receives the state DataFrame and the production line and returns the availability, run_time, and down_time.

In [ ]:
# suggested steps
# select state data from one line
ind_line_1 = df_state['Line'] == 'line_1'
df_state_line_1 = df_state[ind_line_1]
df_state_line_1

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:00:00.000000,state,idle,line_1,0.892188
2022-07-01 00:00:53.531259,state,idle,line_1,1.409600
2022-07-01 00:02:18.107264,state,idle,line_1,1.324097
2022-07-01 00:03:37.553060,state,normal,line_1,12.179597
2022-07-01 00:15:48.328854,state,idle,line_1,0.697680
...,...,...,...,...
2022-07-29 23:39:32.562025,state,idle,line_1,1.013330
2022-07-29 23:40:33.361839,state,normal,line_1,5.198722
2022-07-29 23:45:45.285136,state,idle,line_1,1.309132


In [ ]:
# select only state normal
ind_normal = df_state_line_1['Value'] == 'normal'
df_state_normal = df_state_line_1[ind_normal]
df_state_normal.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:03:37.553060,state,normal,line_1,12.179597
2022-07-01 00:17:15.964082,state,normal,line_1,6.347260
2022-07-01 00:23:36.799710,state,normal,line_1,5.812493
2022-07-01 00:30:39.753513,state,normal,line_1,14.820646
2022-07-01 00:45:28.992274,state,normal,line_1,6.682509


In [ ]:
# evaluate total time as state normal (run time)
run_time = sum(df_state_normal['Time'])
run_time

35141.80224121673

In [ ]:
# select all states except normal
df_state_not_normal = df_state_line_1[~ind_normal]
df_state_not_normal.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:00:00.000000,state,idle,line_1,0.892188
2022-07-01 00:00:53.531259,state,idle,line_1,1.409600
2022-07-01 00:02:18.107264,state,idle,line_1,1.324097
2022-07-01 00:15:48.328854,state,idle,line_1,0.697680
2022-07-01 00:16:30.189664,state,idle,line_1,0.762907


In [ ]:
# evaluate down time
down_time = sum(df_state_not_normal['Time'])
down_time

6620.0121588499915

In [ ]:
# evaluate availability
availability_value = run_time / (run_time + down_time)
availability_value

0.8414816919726693

In [ ]:
# create function with all steps
def availability(df_state, line):
    """
    Function to evaluate line availability.
    """
    # select state data from one line
    ind_line_1 = df_state['Line'] == line
    df_state_line_1 = df_state[ind_line_1]

    # evaluate run time
    ind_normal = df_state_line_1['Value'] == 'normal'
    df_state_normal = df_state_line_1[ind_normal]
    run_time = sum(df_state_normal['Time'])

    # evaluate down time
    df_state_not_normal = df_state_line_1[~ind_normal]
    down_time = sum(df_state_not_normal['Time'])

    # return availability, run time, down time
    return run_time / (run_time + down_time), run_time, down_time

In [ ]:
# test for line_2
availability(df_state, 'line_2')

(0.6646911785927448, 27757.590589649983, 14002.540255500004)

### Quality

quality = successes / (successes + failures) – The machine's percentage of successfully filled units over the specified time intervals.

Aim: build a function that receives the counter DataFrame and the production line and returns the quality, number of successes, and number of failures

In [ ]:
# select state data from one line
ind_line_1 = df_count['Line'] == 'line_1'
df_count_line_1 = df_count[ind_line_1]
df_count_line_1.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:03:37.553060,good_count,37,line_1,12.179597
2022-07-01 00:03:37.553060,bad_count,1,line_1,12.179597
2022-07-01 00:17:15.964082,good_count,18,line_1,6.347260
2022-07-01 00:17:15.964082,bad_count,0,line_1,6.347260
2022-07-01 00:23:36.799710,good_count,20,line_1,5.812493


In [ ]:
# select only units good
ind_good=  df_count_line_1['Measurement'] == 'good_count'
df_good = df_count_line_1[ind_good]
df_good.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:03:37.553060,good_count,37,line_1,12.179597
2022-07-01 00:17:15.964082,good_count,18,line_1,6.347260
2022-07-01 00:23:36.799710,good_count,20,line_1,5.812493
2022-07-01 00:30:39.753513,good_count,44,line_1,14.820646
2022-07-01 00:45:28.992274,good_count,22,line_1,6.682509


In [ ]:
# count the total of good units
successes = df_good['Value'].sum()
successes

106651

In [ ]:
# select only units bad
ind_bad = df_count_line_1['Measurement'] == 'bad_count'
df_bad = df_count_line_1[ind_bad]
df_bad.head()

,Measurement,Value,Line,Time
Timestamp,,,,
2022-07-01 00:03:37.553060,bad_count,1,line_1,12.179597
2022-07-01 00:17:15.964082,bad_count,0,line_1,6.347260
2022-07-01 00:23:36.799710,bad_count,0,line_1,5.812493
2022-07-01 00:30:39.753513,bad_count,1,line_1,14.820646
2022-07-01 00:45:28.992274,bad_count,0,line_1,6.682509


In [ ]:
# count the total of bad units
failures = df_bad['Value'].sum()
failures

2119

In [ ]:
# evaluate quality
quality_value = successes / (successes + failures)
quality_value

0.9805185253286752

In [ ]:
# create function with all steps
def quality(df_count, line):
    # select count data from one line
    ind_line_1 = df_count['Line'] == line
    df_count_line_1 = df_count[ind_line_1]

    # evaluate successes
    ind_good = df_count_line_1['Measurement'] == 'good_count'
    df_good = df_count_line_1[ind_good]  
    successes = df_good['Value'].sum()

    # evaluate failures
    ind_bad = df_count_line_1['Measurement'] == 'bad_count'
    df_bad = df_count_line_1[ind_bad]
    failures = df_bad['Value'].sum()

    # return quality, successes, failures
    return successes / (successes + failures), successes, failures

In [ ]:
# test for line_2
quality(df_count, 'line_2')

(0.9474229474229474, 69502, 3857)

### Performance

performance = ((successes + failures) / run_time) / ideal_run_rate – The machine's performance over the specified time interval as a percentage out of the ideal run rate (in seconds) for your process.

Aim: build a function that receives the number of successes, number of failures, run_time, and the ideal_run_rate and returns the performance

In [ ]:
# evaluate real_run_rate
real_run_rate = (successes + failures) / run_time # units per minute
real_run_rate

3.0951742102864337

In [ ]:
# given ideal_run_rate, evaluate performance
ideal_run_rate = 3.4
performance_value = real_run_rate / ideal_run_rate
performance_value

0.9103453559665982

In [ ]:
# create function with all steps
def performance(successes, failures, run_time, ideal_run_rate):
    # evaluate real_run_rate
    real_run_rate = (successes + failures) / run_time

    # return performance
    return real_run_rate / ideal_run_rate

In [ ]:
# test for ideal_run_rate = 3.4
performance(successes, failures, run_time, 3.4)

0.9103453559665982

### OEE

oee = availability_value * quality_value * performance_value

Aim: build a function that joins all functions above and receives the df_state, df_count, ideal_run_rate, and line and returns the OEE.

In [ ]:
# evaluate OEE based on the previous results
availability_value * quality_value * performance_value

0.7511153820084139

In [ ]:
# create a function that runs all previous functions
def oee(df_state, df_count, ideal_run_rate, line):
    availability_value, run_time, down_time = availability(df_state, line)
    quality_value, successes, failures = quality(df_count, line)
    performance_value = performance(successes, failures, run_time, ideal_run_rate)

    return availability_value * quality_value * performance_value, availability_value, quality_value, performance_value

In [ ]:
# test the function for ideal_run_rate = 3.4 and line 2
res = oee(df_state, df_count, 3.4, 'line_2')
print(f'OEE: {res[0]}, Availability: {res[1]}, Quality: {res[2]}, Performance: {res[3]}')

OEE: 0.4895043260683762, Availability: 0.6646911785927448, Quality: 0.9474229474229474, Performance: 0.7773072522596172


# Possible improvements


* Include period selection in the function, so it would be possible to evaluate OEE by day, for example.
* Calculate OEE for all available lines automatically.